In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm import Code, Computer
from aiida.orm.code import Code
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.implementation.django.node import Node
from aiida.orm.calculation.work import WorkCalculation
from aiida.orm.data.base import Int, Float, Str, Bool
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.work.run import submit

import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from IPython.display import display, clear_output, HTML
    
from apps.surfaces.structure_browser import StructureBrowser
from apps.surfaces.slab.dftwork import DFTGeoOptWorkChain

from tempfile import NamedTemporaryFile
import ase.io
import ase.build
from base64 import b64encode

from aiida_cp2k.calculations import Cp2kCalculation

from pprint import pprint, pformat

In [ ]:
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

In [ ]:
def html_table_row(cells):
    ret = '<tr>'
    for c in cells:
        ret += '<td>{}</td>'.format(c)
    ret += '</tr>'
    return ret

# Calculate Adsorption energy

# Step 1: Select structure - molecule on slab

In [ ]:
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        viewer.clear_representations()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    node = struct_browser.results.value
    if not node:
        return
    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()

    spin_ups = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("1")])
    spin_downs = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("2")])

    viewer.add_representation('ball+stick', selection="@"+spin_ups, color='red', aspectRatio=3.0, opacity=0.4)
    viewer.add_representation('ball+stick', selection="@"+spin_downs, color='green', aspectRatio=3.0, opacity=0.4)

In [ ]:
def on_struct_change(c):
    refresh_structure_view()

    try:
        ase_struct = struct_browser.results.value.get_ase()
        first_slab_atom = np.argwhere(ase_struct.numbers == 79)[0, 0]
        last_slab_atom = len(ase_struct.numbers)
        mol = ase_struct[:first_slab_atom]
        slab = ase_struct[first_slab_atom:]
        mol_formula = mol.get_chemical_formula()
        slab_formula = slab.get_chemical_formula('metal')
        candidates = find_existing(mol, slab, struct_browser.results.value)
    except AttributeError:
        candidates = []  
    except IndexError:
        candidates = []  
        with rendered_search:
            clear_output()
            print "Not a gold slab."
        
    with rendered_search:
        clear_output()
        render_search(candidates)
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()

clear_output()
display(ipw.VBox([struct_browser, viewer]))

In [ ]:
# query AiiDA database for Computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'cp2k'})

all_computers = OrderedDict()
all_computers['Please select a computer'] = False
for match in qb.all():
    label = match[0]
    code = match[1]
    all_computers[label] = code

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer", options=all_computers,
                            style=style, layout=layout)

text_fixed_atoms = ipw.Text(placeholder='1..12 25 110',
                            description='Fixed Atoms',
                            disabled=True,
                            style=style, layout=layout)

display(ipw.VBox([drop_computer, text_fixed_atoms]))

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return
            
        cp2k_code = drop_computer.value
        struct = struct_browser.results.value
        
        cp2kcalc = struct.get_inputs(node_type=Cp2kCalculation)[0]
        wc = cp2kcalc.get_inputs(node_type=WorkCalculation)[0]
        old_inputs = wc.get_inputs_dict()
        
        if not old_inputs['calc_type'] == 'Full DFT':
            print("Warning: Are you sure the selected structure was calculated using Full DFT?")
            return
        
        max_force = old_inputs['max_force']
        mgrid_cutoff = old_inputs['mgrid_cutoff']
        vdw_switch = old_inputs['vdw_switch']
        
        slab_n = slab.get_atomic_numbers()
        slab_h = np.count_nonzero(slab_n==1)
        slab_three_rows = slab_h*3

        first_fixed_atom = len(atoms)-slab_three_rows
        last_fixed_atom = len(atoms)
        fixed_atoms = '{}..{}'.format(first_fixed_atom, last_fixed_atom)
        
        # Submit Mol
        outputs = submit(DFTGeoOptWorkChain,
                         cp2k_code=cp2k_code,
                         structure=StructureData(ase=mol),
                         max_force=max_force,
                         mgrid_cutoff=mgrid_cutoff,
                         vdw_switch=vdw_switch,
                         fixed_atoms=Str(''))
        # Submit Slab
        outputs = submit(DFTGeoOptWorkChain,
                         cp2k_code=cp2k_code,
                         structure=StructureData(ase=slab),
                         max_force=max_force,
                         mgrid_cutoff=mgrid_cutoff,
                         vdw_switch=vdw_switch,
                         fixed_atoms=Str(fixed_atoms))        
        print(outputs)



btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)

# Step 2: Choose from calculated structures

In [ ]:
def find_existing(mol, slab, molonslab):
    qb = QueryBuilder()
    qb.append(StructureData,
              filters = {
                  'extras': {'has_key': 'formula'},
                  'or': [
                      {'extras.formula': {'==': mol.get_chemical_formula()}},
                      {'extras.formula': {'==': slab.get_chemical_formula()}},
                      {'extras.formula': {'==': slab.get_chemical_formula('metal')}},
                  ]
              },
              project=['extras.formula', '*', 'id'],
              tag = 'structure',
    )
    qb.append(WorkCalculation,
              project='*',
              output_of='structure',
              tag='workcalc')

    qb.append(Cp2kCalculation,
              project=['label', 'id'],
              output_of='workcalc',
#              filters = {
#                  'label': {'==': 'dft - slab_geo_opt'}
#              },
              tag='cp2kcalc'
    )

    qb.append(ParameterData,
              project='attributes.FORCE_EVAL.DFT',
              input_of='cp2kcalc',
              filters = {
                  'attributes': {'!has_key': 'additional_retrieve_list'},
                  'attributes.FORCE_EVAL': {'has_key': 'DFT'}
              },
              tag='input'
    )

    qb.append(ParameterData,
              project='attributes.energy',
              output_of='cp2kcalc',
              filters = {
                  'attributes.exceeded_walltime': {'==': False}
              },
              tag='output'
    )
    qb.order_by({'structure': {'id': 'asc'}})

    candidate_molecules = []
    molonslab_cp2k = molonslab.get_inputs(Cp2kCalculation)[0]

    candidate_molecules.append({
        'pk': molonslab.pk,
        'formula': molonslab.get_extra('formula'),
        'calc_label': molonslab_cp2k.label,
        'input': molonslab_cp2k.get_inputs(WorkCalculation)[0].get_inputs_dict(),
        'energy': molonslab_cp2k.get_outputs_dict()['output_parameters'].get_dict()['energy'],
        'thumbnail': render_thumbnail(molonslab.get_ase())
    })
    for m in qb.iterdict():
        candidate_molecules.append({
            'pk': m['structure']['id'],
            'formula': m['structure']['extras.formula'],
            'calc_label': m['cp2kcalc']['label'],
            'input': m['workcalc']['*'].get_inputs_dict(),
            'energy': m['output']['attributes.energy'],
            'thumbnail': render_thumbnail(m['structure']['*'].get_ase())
        })
    
    return candidate_molecules

In [ ]:
def render_search(candidates):
    html = '''
        <style>
            #search td, th {text-align:left !important; vertical-align:top;}
            #search tr {border-bottom: 1px solid #777;}
        </style>
    '''
    html += '<table id="search">'
    html += '''<tr>
                <th>pk</th>
                <th>Formula</th>
                <th>Calc Label</th>
                <th>Input</th>
                <th>Energy</th>
                <th></th>
               </tr>'''
    for c in candidates:
        html += html_table_row([c['pk'],
                                c['formula'],
                                c['calc_label'],
                                '<pre>{}</pre>'.format(pformat(c['input'])),
                                c['energy'],
                                '<img width="100px" src="data:image/png;base64,{}">'.format(c['thumbnail'])])
    html += '</table>'
    display(HTML(html))

In [ ]:
rendered_search = ipw.Output()
display(rendered_search)